## Очистка и подготовка данных

### Выгрузка библиотек

In [ ]:
# ! pip install datasets --progress-bar off

In [5]:
from datasets import load_dataset
import pandas as pd

### Выгрузка данных и первичная отладка

In [3]:
dataset = load_dataset("d0rj/geo-reviews-dataset-2023")
work_data = pd.DataFrame(dataset['train'])

In [ ]:
work_data.head()

In [ ]:
work_data = work_data.dropna(subset=['text', 'name_ru', 'rating'])
work_data = work_data.drop_duplicates(subset=['text']).reset_index(drop=True)
work_data['text'] = work_data['text'].str.replace('\\n', ' ')

work_data.info()

In [ ]:
work_data.head()

In [ ]:
import re

''' Очистка текста от лишних элементов '''
def clean_review_text(text: str) -> str:
    text = text.lower()                         # <-- приводим к нижнему регистру
    text = re.sub(r"<[^>]+>", "", text)         # <-- удаляем HTML-теги
    text = re.sub(r"[^\w\s,.!?()]+", "", text)  # <-- удаляем спецсимволы, кроме пунктуации и скобок
    text = re.sub(r"\)\)+", " ", text)          # <-- заменяем смайлики вида "))" на пробел
    text = re.sub(r"\s+", " ", text).strip()    # <-- убираем лишние пробелы
    text = re.sub(r"[\n\r]+", " ", text)        # <-- заменяем переносы строк на пробелы
    return text

''' Очистка столбца рубрик для разделения по тематикам '''
def clean_rubrics(rubrics_: str) -> str:
    return rubrics_.lower().strip()

''' Очистка наименования объекта '''
def clean_name_ru(name_ru: str) -> str:
    return name_ru.lower().strip()

''' Очистка адреса '''
def clean_address(address: str) -> str:
    address = address.strip()
    return re.sub(r"\s+", " ", address)


# Применяем ко всем столбцам в датасете очистку
work_data['text'] = work_data['text'].apply(clean_review_text)
work_data['rubrics'] = work_data['rubrics'].apply(clean_rubrics)
work_data['name_ru'] = work_data['name_ru'].apply(clean_name_ru)
work_data['address'] = work_data['address'].apply(clean_address)

# Преобразование рейтинга в числовой формат
work_data['rating'] = work_data['rating'].astype(float)

# Удаление строк с пропущенными значениями
work_data.dropna(subset=['address', 'name_ru', 'rubrics', 'rating', 'text'], inplace=True)
work_data.info()

In [ ]:
work_data.head()

### Преобразование данных, интеграция с поиском ключевых слов (выбрано в качестве шаблона 3)

In [9]:
# Импорт библиотек
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords


# Загружаем стоп-слова
nltk.download('stopwords', quiet=True)
russian_stopwords = stopwords.words('russian')


# Функция для извлечения трех ключевых слов
def extract_keywords(text):
    text = text.strip()
    if not text:            # <-- проверка на пустую строку
        return ''

    vectorizer = CountVectorizer(stop_words=russian_stopwords, max_features=3)

    try:
        X = vectorizer.fit_transform([text])
        # Извлекаем ключевые слова
        keywords = vectorizer.get_feature_names_out()
        return ', '.join(keywords)
    except:
        return ''

In [10]:
work_data['key_words'] = work_data['text'].apply(extract_keywords)

In [ ]:
work_data.info()

In [ ]:
# Удаление данных без ключевых слов
weird_data = work_data[work_data['key_words']=='']
weird_data.info()

In [ ]:
work_data = work_data[work_data['key_words']!='']
work_data.info()

In [ ]:
work_data.to_csv('work_data.csv', index=False)
print("Данные успешно очищены и сохранены в 'work_data.csv'.")

## Начало работы с моделью: GPT2

### Выгрузка библиотек

In [15]:
!pip install -q accelerate --progress-bar off
!pip install -q peft --progress-bar off
!pip install -q bitsandbytes --progress-bar off
!pip install -q transformers --progress-bar off
!pip install -q trl --progress-bar off

In [ ]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoModelForSequenceClassification,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer

### Настройка отдельных элементов для Нейронной Сети

#### Параметры модели

In [17]:
'''
Настраиваем метод квантизации модели с использованием bitsandbytes для ускорения обучения и вывода
-  load_in_4bit                 <-- загрузка модель в режиме 4-битной точности
-  bnb_4bit_use_double_quant    <-- вложенная квантизация для 4-битной модели
-  bnb_4bit_quant_type          <-- тип данных для квантизации 4-битной модели
-  bnb_4bit_compute_dtype       <-- тип данных для вычислений 4-битной модели
'''

def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    bnb_config = BitsAndBytesConfig(
                                    load_in_4bit = load_in_4bit,
                                    bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
                                    bnb_4bit_quant_type = bnb_4bit_quant_type,
                                    bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
                                    )
    return bnb_config

#### Параметры выгрузки модели

In [18]:
import time

''' 
Загружает модель и токенизатор модели.
- model_name    <-- имя модели из библиотеки Hugging Face
- bnb_config    <-- конфигурация Bitsandbytes
'''
def load_model(model_name, bnb_config):

    # Получает количество доступных GPU и задает максимальный объем памяти
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'
    
    time.sleep(1)
    
     # Загружает модель
    model = AutoModelForCausalLM.from_pretrained(
                                                model_name,
                                                quantization_config = bnb_config,
                                                device_map = 'auto',        # <-- автоматически распределяет модель на доступные ресурсы
                                                max_memory = {i: max_memory for i in range(n_gpus)},
                                                )

    time.sleep(1)
    
    # Загружает токенизатор модели с использованием токена аутентификации пользователя
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = False)

     # Устанавливает токен для заполнения (padding) как токен конца предложения (EOS)
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

#### Выбор модели

In [19]:
''' 
Параметры transformers:
список опробованных обученных моделей из Hugging Face Hub для загрузки и тонкой настройки 
'''

# model_name = "meta-llama/Llama-2-7b-hf"
# model_name = "daryl149/llama-2-7b-chat-hf"
# model_name = "OpenBuddy/openbuddy-llama2-13b-v8.1-fp16"
# model_name = "OpenBuddy/openbuddy-llama-7b-v4-fp16"
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"    # <-- модель, на которой на данный момент остановили выбор
#model_name = "rccmsu/ruadapt_llama2_7b_v0.1"


''' 
Параметры bitsandbytes
* Bitsandbytes — это библиотека, которая предоставляет инструменты для эффективного использования моделей машинного обучения 
  с низкой точностью чисел, таких как 8-битные и 4-битные представления, что позволяет значительно уменьшить объем используемой 
  памяти и ускорить вычисления. Это особенно полезно при работе с крупными языковыми моделями, которые требуют значительных вычислительных ресурсов.
'''

# Активировать загрузку базовой модели с 4-битной точностью
load_in_4bit = True

# Активировать вложенную квантизацию для 4-битных базовых моделей (двойная квантизация)
bnb_4bit_use_double_quant = True

# Тип квантизации (fp4 или nf4)
bnb_4bit_quant_type = "nf4"

# Тип данных для вычислений для 4-битных базовых моделей
bnb_4bit_compute_dtype = torch.float32

#### Загрузка модели

In [20]:
# Загрузка модели из Hugging Face Hub с использованием имени модели и конфигурации bitsandbytes
bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

In [ ]:
model, tokenizer = load_model(model_name, bnb_config)

### Загрузка датасета  
Будем использовать заранее подготовленный датасет с инструкциями.  

В данном случае это csv-таблица с 3 колонками:  

* Инструкция  
* Текст  
* Класс  

Мы будем использовать стандартный генератор датасета типа `csv` (потому что у нас файл CSV). Аналогично здесь мог бы быть файл JSON и типа датасет `json`. По умолчанию все записи относятся к разделению `train`, который мы получим с помощью параметра `split`.

In [ ]:
dataset = load_dataset('csv', data_files='work_data.csv', split='train')

In [ ]:
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Функция `load_dataset` преобразует файл CSV в словарь промтов. Мы можем просмотреть объекты, используя случайный индекс.

In [ ]:
dataset[randrange(len(dataset))]

### Создание шаблона промта  

In [25]:
''' Объединение шаблона с неизменными строками <=> Создание промта для модели на основе содержания промтов в датасте '''
def create_prompt_formats(sample):

    # Разделяет рубрики, предполагая, что они разделены точками с запятой
    categories = sample['rubrics'].split(';')           
    categories = [cat.strip() for cat in categories]    # <-- удаляет пробелы в начале и конце каждой рубрики
    categories_str = ', '.join(categories)              # <-- объединяет рубрики в строку через запятую
    categories = f"Категории:\n{categories_str}"        # <-- формирует строку с заголовком "Категории"

    # Формирует строки
    rating = f"Рейтинг:\n{sample['rating']}"
    keywords = f"Ключевые слова:\n{sample['key_words']}"
    rewiev = f"Отзыв:\n{sample['text']}"

    # Создает список элементов шаблона подсказки
    parts = [part for part in [categories, rating, keywords, rewiev] if part]

    # Объединяет элементы шаблона подсказки в одну строку для создания шаблона подсказки
    formatted_prompt = "\n\n".join(parts)

    # Сохраняет отформатированный шаблон подсказки в новом ключе "prompt"
    sample["prompt"] = formatted_prompt

    return sample

In [ ]:
create_prompt_formats(dataset[randrange(len(dataset))])

In [ ]:
type(dataset)

In [ ]:
dataset.column_names

### Получение максимальной длины последовательности предобученной модели

In [29]:
''' 
Извлечение максимальной длины токенов из конфигурации модели
* Эта функция извлечет конфигурацию модели и попытается найти максимальную длину последовательности из одного из нескольких ключей конфигурации, 
  которые могут ее содержать. Если максимальная длина последовательности не найдена, по умолчанию она будет равна 1024. 
  Мы будем использовать максимальную длину последовательности во время предварительной обработки датасета, чтобы удалить записи, 
  которые превышают эту длину контекста, поскольку предварительно обученная модель не примет их.
'''
def get_max_length(model):

    # Получает конфигурацию модели
    conf = model.config
    
    # Инициализирует переменную "max_length" для хранения максимальной длины последовательности как пустую
    max_length = None
    
    # Ищет максимальную длину последовательности в конфигурации модели и сохраняет её в "max_length", если найдена
    for length_setting in ['n_positions', 'max_position_embeddings', 'seq_length']:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
        
    # Устанавливает "max_length" в 1024 (значение по умолчанию), если максимальная длина последовательности не найдена в конфигурации модели
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
        
    return max_length

#### Токенизирование батчей  

Функция `preprocess_batch` будет токенизировать входящий (`batch`) используя  `tokenizer`.  

Мы устанавливаем параметр максимальной длины последовательности  `max_length`, от которго будет зависеть паддинг либо обрезка данных.

In [30]:
'''
Токенизация пакета данных
- batch       <-- пакет данных
- tokenizer   <-- токенизатор модели
- max_length  <-- максимальное количество токенов, которое должен возвращать токенизатор
'''
def preprocess_batch(batch, tokenizer, max_length):
    return tokenizer(
                    batch['prompt'],
                    max_length = max_length,
                    truncation = True,
                    )

#### Предобработка датасета  

1. Создание промтов через функцию `create_prompt_formats`.  

2. Токенизирование батчей через функцию `preprocess_batch`, удаление исходных колонок (instruction, input, output, text).  

3. Фильтрация итоговых промтов по максимальной длине в токенах.  

4. Перемешивание датасета (shuffle) с инициализацией random seed.

In [31]:
'''
Токенизирует набор данных для тонкой настройки.
- tokenizer (AutoTokenizer) <-- токенизатор модели
- param max_length (int)    <-- максимальное количество токенов, которое должен возвращать токенизатор
- seed:                     <-- случайное зерно для воспроизводимости
- dataset (str)             <-- набор данных с инструкциями
'''
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):

    # Добавляет шаблон подсказки к каждому образцу
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Применяет предобработку к каждому пакету набора данных и удаляет ненужные поля
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
                            _preprocessing_function,
                            batched = True,
                            remove_columns = ['address', 'name_ru', 'rating', 'rubrics', 'text','key_words'],
                            )

    # Фильтрует образцы, у которых "input_ids" превышает "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Перемешивает набор данных
    dataset = dataset.shuffle(seed = seed)
    return dataset

In [ ]:
# RD Для фиксации случайного элемента и возможности воссоздать "эксперимент"
seed = 11111

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

Вот так выглядит теперь датасет, состоящий из токенов

In [ ]:
print(preprocessed_dataset)

In [ ]:
print(preprocessed_dataset[0])

## Основной этап обучения Модели

### Создание конфигурации PEFT

Подход PEFT позволяет тюнить небольшое количество дополнительных параметров модели, одновременно замораживая большинство параметров предварительно обученных LLM, значительно снижая затраты на вычисления и хранение. Это также помогает обеспечить переносимость: пользователи могут настраивать модели с помощью методов PEFT, чтобы получить LoRa-модули в размером в несколько МБ.


Воспользуемся библиотекой `peft` из Hugging Face.

Существует несколько методов PEFT. Мы будем использовать QLoRA, применяя класс `LoraConfig` из библиотеки  `peft`.

QLoRA квантует модель в 4 бита, затем замораживает веса основной модели и добавляет две матрицы обучаемых мараметров. Во время тюнигна, QLoRA пробрасывает градиент через замороженную часть общей модели.

Обновляются только веса LoRa-модуля.

In [35]:
'''
Создает конфигурацию для параметрически-эффективной тонкой настройки модели.
- r                 <-- размерность внимания LoRA
- lora_alpha        <-- параметр альфа для масштабирования LoRA
- target_modules    <-- имена модулей, к которым будет применяться LoRA
- lora_dropout      <-- вероятность применения Dropout в слоях LoRA
- bias              <-- указывает, следует ли обучать параметры смещения (bias)
- task_type         <-- тип задачи, для которой настраивается модель
'''
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    config = LoraConfig(
                        r = r,
                        lora_alpha = lora_alpha,
                        target_modules = target_modules,
                        lora_dropout = lora_dropout,
                        bias = bias,
                        task_type = task_type,
                        )
    return config

#### Поиск модулей для LoRA

Функция `find_all_linear_names` предназначен для поиска слоёв оригинальной сети, для которых будет применяться LoRa.


Функция получает названия слоёв через `model.named_modules()`.

In [36]:
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit             # <-- определяет класс 4-битных линейных модулей
    lora_module_names = set()           # <-- создает множество для хранения имен модулей LoRA

    for name, module in model.named_modules():
        if isinstance(module, cls):                                             # <-- проверяет, является ли модуль экземпляром Linear4bit
            names = name.split('.')                                             # <-- разделяет полное имя модуля на части
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])   # <-- добавляет имя модуля в множество

    # Удаляет 'lm_head', если он есть в списке, так как обычно не применяется LoRA
    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
        
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)      # <-- возвращает список имен модулей LoRA

#### Подсчёт обучаемых параметров

Функция `print_trainable_parameters` предназначена для расчёта количества обучаемых параметров в `model.named_parameters()`.

In [37]:
def print_trainable_parameters(model, use_4bit = False):
    trainable_params = 0    # <-- переменная для подсчета количества обучаемых параметров
    all_param = 0           # <-- переменная для подсчета общего количества параметров

    for _, param in model.named_parameters():
        # Получает количество элементов в параметре
        num_params = param.numel()      
        
        # Использует альтернативное количество элементов, если доступно
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        
        all_param += num_params
        
        # Увеличивает количество обучаемых параметров, если они требуют градиента
        if param.requires_grad:
            trainable_params += num_params

    # Делит количество обучаемых параметров на 2, если используется 4-битная точность
    if use_4bit:
        trainable_params /= 2

    print(f"All Parameters: {all_param:,d} || \
            Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}")

### Fine-tuning предобученной модели

Функция `fine_tune` оборачивает описанные выше модули и запускает их:

1. Разрешить сохранение градиентов (gradient checkpointing) для уменьшения использования памяти во время тюнинга.  

2. Использование функции `prepare_model_for_kbit_training` из PEFT для подготови модели к тюнингу.  

3. Вызов `find_all_linear_names` для получения названий слоёв сети для применения LoRA.  

4. Создание конфигурации LoRA через вызов функции `create_peft_config`.  

5. Оборачивание базовой модели с Hugging Face model для тюнинга через PEFT путём вызова функции `get_peft_model`.  

6. Печать обучаемых параметров.  


Для обучения мы инициализируем объект `Trainer()` внутри функции `fine_tune`, который требует:


* `per_device_train_batch_size` — размер батча на обучении.

* `gradient_accumulation_steps` — количество шагов, для которых необходимо накопить градиенты перед выполнением обратного прохода.

* `warmup_steps` — количество шагов линейного увеличения скорости обучения от 0 до `learning_rate`.  

* `max_steps`: количество шагов обучения.  

* `learning_rate`: начальный  learning rate для Adam.  

* `fp16`: использовать ли 16-bit (mixed) обучение вместо  32-bit.  

* `logging_steps`: количество шагов между двумя логированиями.  

* `output_dir`: папка для сохранения логов и модели.  

* `optim`: оптимизатор для обучения.

In [38]:
'''
Подготовка и выполнение тонкой настройки предварительно обученной модели.
- model        <-- предварительно обученная модель от Hugging Face
- tokenizer    <-- токенизатор для модели
- dataset      <-- предварительно обработанный набор данных для обучения
'''
def fine_tune(model, tokenizer, dataset, lora_r, lora_alpha, lora_dropout, bias,
                task_type, per_device_train_batch_size, gradient_accumulation_steps,
                    warmup_steps, max_steps, learning_rate, fp16, logging_steps, output_dir, optim):

    # Включает градиентное контрольное сохранение для уменьшения использования памяти при тонкой настройке
    model.gradient_checkpointing_enable()

    # Подготавливает модель для обучения с использованием квантизации
    model = prepare_model_for_kbit_training(model)

    # Получает имена модулей LoRA
    target_modules = find_all_linear_names(model)

    # Создает конфигурацию PEFT для этих модулей и оборачивает модель в PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Выводит информацию о проценте обучаемых параметров
    print_trainable_parameters(model)

    # Параметры обучения
    trainer = Trainer(
                        model = model,
                        train_dataset = dataset,
                        args = TrainingArguments(
                                                per_device_train_batch_size = per_device_train_batch_size,
                                                gradient_accumulation_steps = gradient_accumulation_steps,
                                                warmup_steps = warmup_steps,
                                                max_steps = max_steps,
                                                learning_rate = learning_rate,
                                                fp16 = fp16,
                                                logging_steps = logging_steps,
                                                output_dir = output_dir,
                                                optim = optim,
                                                report_to='tensorboard'                         # <-- логирование результатов в TensorBoard
                                                ),
                        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False) # <--  # подготовка данных для языкового моделирования без маскирования
                        )

    # Отключает использование кэша
    model.config.use_cache = False

    do_train = True

    # Запускает обучение и логирует метрики
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)   # <-- логирует метрики обучения
        trainer.save_metrics("train", metrics)  # <-- сохраняет метрики обучения
        trainer.save_state()                    # <-- сохраняет состояние обучения
        print(metrics)

    # Сохраняет последнюю контрольную точку модели
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    # Освобождает память для объединения весов
    del model
    del trainer
    torch.cuda.empty_cache()    # <-- ВАЖНАЯ ШТУКА: очищает кэш GPU!!!!
    

### Использование парамертов QLoRa для обучения

In [39]:
# Размерность внимания LoRA
lora_r = 16

# Параметр альфа для масштабирования LoRA
lora_alpha = 64

# Вероятность dropout для слоев LoRA
lora_dropout = 0.1

# Смещение
bias = "none"

# Тип задачи
task_type = "CAUSAL_LM"

### Настройка гиперпараметров

In [40]:
# Директория для хранения предсказаний модели и контрольных точек
output_dir = "./results"

# Размер пакета данных на одну GPU для обучения (собственно, в данном случае одна GPU и есть)
per_device_train_batch_size = 1

# Количество шагов обновления для накопления градиентов
gradient_accumulation_steps = 4

# Начальная скорость обучения (оптимизатор AdamW)
learning_rate = 2e-5

# Используемый оптимизатор
optim = "paged_adamw_32bit"

# Количество шагов обучения (переопределяет num_train_epochs)
max_steps = 40000 #0

# Линейное увеличение шагов от 0 до learning_rate
warmup_steps = 100

# Включить обучение с использованием fp16
fp16 = True

# Логирование каждые X шагов обновления
logging_steps = 1

### Запуск обучения модели

In [ ]:
fine_tune(model,
            tokenizer,
            preprocessed_dataset,
            lora_r,
            lora_alpha,
            lora_dropout,
            bias,
            task_type,
            per_device_train_batch_size,
            gradient_accumulation_steps,
            warmup_steps,
            max_steps,
            learning_rate,
            fp16,
            logging_steps,
            output_dir,
            optim)

## Результаты и тестирование

### Выгрузка обученной модели

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

# Сразу указываем, что девайс - GPU
device = 'cuda'

# Укажите путь к контрольной точке с LoRA-адаптерами
checkpoint_path = "results/checkpoint-40000"

# Загрузка базовой модели
base_model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Загрузка конфигурации LoRA-адаптера
peft_config = PeftConfig.from_pretrained(checkpoint_path)

# Объединение базовой модели с LoRA-адаптером
model = PeftModel.from_pretrained(base_model, checkpoint_path)

# Перемещение модели на GPU
model = model.to(device)

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

c:\Users\pront\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\tuners\lora\layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


### Выгрузка данных

In [2]:
import pandas as pd

# Обработанные ранее данные
data_path = 'work_data.csv'

# Загрузка данных
data = pd.read_csv(data_path)

# Просмотр первых 5 строк
print(data.head())

                                             address             name_ru  \
0  Екатеринбург, ул. Московская / ул. Волгоградск...  московский квартал   
1  Московская область, Электросталь, проспект Лен...   продукты ермолино   
2  Краснодар, Прикубанский внутригородской округ,...             limefit   
3   Санкт-Петербург, проспект Энгельса, 111, корп. 1        snow-express   
4                  Тверь, Волоколамский проспект, 39  студия beauty brow   

   rating                                            rubrics  \
0     3.0                                     жилой комплекс   
1     5.0  магазин продуктов;продукты глубокой заморозки;...   
2     1.0                                        фитнес-клуб   
3     4.0        пункт проката;прокат велосипедов;сапсёрфинг   
4     5.0  салон красоты;визажисты, стилисты;салон бровей...   

                                                text  \
0  московский квартал 2. шумно летом по ночам дик...   
1  замечательная сеть магазинов в общем, хорош

In [3]:
# Разделение значений в столбце rubrics
data['rubrics_split'] = data['rubrics'].str.split(';')

# Уникальные значения категорий
unique_rubrics = pd.Series([rubric for sublist in data['rubrics_split'].dropna() for rubric in sublist]).unique()
print('Уникальные рубрики:')
for rubric in unique_rubrics:
    print('-', rubric)

Уникальные рубрики:
- жилой комплекс
- магазин продуктов
- продукты глубокой заморозки
- магазин мяса, колбас
- фитнес-клуб
- пункт проката
- прокат велосипедов
- сапсёрфинг
- салон красоты
- визажисты, стилисты
- салон бровей и ресниц
- оператор сотовой связи
- интернет-провайдер
- кафе
- вейп-шоп
- магазин табака и курительных принадлежностей
- кофейня
- магазин обуви
- ювелирный магазин
- детские игрушки и игры
- пляж
- кондитерская
- ресторан
- магазин подарков и сувениров
- музыкальный магазин
- гостиница
- быстрое питание
- доставка еды и обедов
- автомойка
- пекарня
- столовая
- компьютерный магазин
- компьютерный ремонт и услуги
- хостел
- магазин мебели
- косметология
- ногтевая студия
- эпиляция
- спа-салон
- магазин цветов
- товары для праздника
- мягкая мебель
- корпусная мебель
- аптека
- автомобильные грузоперевозки
- перевозка негабаритных грузов
- супермаркет
- фотоуслуги
- художественный салон
- массажный салон
- авиакомпания
- железнодорожные и авиабилеты
- курсы инос

In [4]:
# Уникальные значения рейтингов
ratings = data['rating'].unique() 
print('Уникальные рейтинги:', ratings)

Уникальные рейтинги: [3. 5. 1. 4. 2. 0.]


In [5]:
# Удаляем цифры и извлекаем уникальные ключевые слова
keywords_split = data['key_words'].str.split(',').explode() 
  
# Фильтруем ключевые слова без цифр
keywords_no_digits = keywords_split[~keywords_split.str.contains(r'\d')].unique()

print('Уникальные ключевые слова без цифр:')
for keyword in keywords_no_digits:
    print('-', keyword)

Уникальные ключевые слова без цифр:
-  обычно
-  окна
- ассортимент
-  высоте
-  сеть
- говорят
-  доверенности
-  номер
- аренды
-  ботинки
-  бывают
- ангелина
-  всё
-  топ
- доступно
-  общение
-  объяснили
-  приемлемые
-  проявлениях
-  атмосфера
-  шоп
- бариста
-  улучшить
-  фирменными
-  очень
-  туда
-  акция
- аквапарком
-  пляж
-  рядом
- вкусная
-  разной
-  разнообразное
- большой
-  отдыхе
-  хочется
- алле
-  посетить
-  спасибо
- очень
-  чаше
-  чашу
- ещё
-  крутые
-  обязательно
- блюдами
-  высокомерием
-  место
-  доставка
-  факту
- езжу
-  такими
-  требовать
- вашему
-  вкусностей
-  вкусу
- безумно
-  мастер
-  такие
- внимательный
-  кафе
-  кухня
- вкусно
-  домашнему
-  дорого
- буду
-  комп
-  стал
- вежливый
-  всем
-  доступные
-  хостела
-  хостелможно
- высоком
-  доступности
-  обслуживание
- вкус
-  внутри
-  выбор
- алкаша
-  пакетиках
-  плохой
- всем
-  изначально
-  мебели
- врачу
-  клинику
-  посещаю
- бесконечно
-  определенного
-  отвратител

### Функция генерации отзывов

In [6]:
def generate_review(category, rating, keywords, max_length=50, temperature=1, top_k=100000, top_p=1.5):
    
    prompt = f"Категория: {category}\nРейтинг: {rating}\nКлючевые слова: {keywords}\nОтзыв:"
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model.generate(
                                inputs.to(device),
                                max_length=inputs.shape[1] + max_length,
                                temperature=temperature,
                                top_k=top_k,
                                top_p=top_p,
                                do_sample=True,
                                num_return_sequences=1,
                                pad_token_id=tokenizer.eos_token_id
                                )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Извлечение только текста отзыва
    review = generated_text.split('Отзыв:')[-1].strip()
    return review

### Функция рандомной генерации

In [9]:
import numpy as np
import random


# Функция для генерации случайных значений
def generate_random_combinations(repeats=20, seed=42):

    random.seed(seed)
    
    results = []
    for _ in range(repeats):
        rubric = np.random.choice(unique_rubrics)
        rating = int(np.random.choice(ratings))
        keyword = np.random.choice(keywords_no_digits)
        
        print(f"Рубрика: {rubric}, Рейтинг: {rating}, Ключевое слово: {keyword}")
        review = generate_review(rubric, rating, keyword)
        print(f"Сгенерированный отзыв: {review}\n")

In [10]:
generate_random_combinations()

Рубрика: гидравлическое и пневматическое оборудование, Рейтинг: 4, Ключевое слово:  стране
Сгенерированный отзыв: великолепная компания! путала инструмент под заказ, конктэйде почти все другое! молодцы! и побороли себя, убрали любые проблемы. как говориться, пойдем до конца!!! нет проблем с транспортировкой и сборкой этого

Рубрика: магазин чая, Рейтинг: 2, Ключевое слово: борисовны
Сгенерированный отзыв: супер большой, просторный магазин, выбор товаров большой, посуда, игрушки, игрушки, торты, детский велосипед, спортивный инвентарь, посуду, реклама машинки борисовны. выбор разный, можно купить чай, рыбу, мореп

Рубрика: теплоснабжение, Рейтинг: 0, Ключевое слово:  кружков
Сгенерированный отзыв: централизованных поставщиковводососоваказать,таклектричество,домики для отдыха от перемен. хороший парк для прогулок. что интересное может значить для простого просмотра зимней ёлки. кружка горячего шоколада,совой или квас,вы

Рубрика: музыкальный клуб, Рейтинг: 5, Ключевое слово:  уплетал
Сге

### Непосредственно сама генерация